In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob
from IPython.display import display
import os
import sys
from datetime import date,timedelta
import json
import numpy as np
import time
from math import cos, sin

In [2]:
def checkEyeError(files):
    df = pd.DataFrame()
    for fi in files: # reading all files
        with open(fi, 'r') as j:
            tmp = json.loads(j.read())
            
            tempdf0 = pd.DataFrame.from_dict(tmp,orient='index').T        
            
            tempdf0[["EyeValidationError.x","EyeValidationError.y","EyeValidationError.z"]] = pd.DataFrame(tempdf0["EyeValidationError"].to_list(), columns=['x', 'y','z'])
            
            if(tempdf0["EyeValidationError.x"][0]<1.5) and (tempdf0["EyeValidationError.y"][0]<1.5):
                df = pd.concat([df, tempdf0], ignore_index=True)
          

    df = df.drop(['IPAddress', 'VRmode',
           'SteeringInputDevice', 'EyeValidationError', 'SeatCalibrationOffset',
           'TrainingDuration', 'ExperimentDuration', 'ApplicationDuration',
           'AverageExperimentFPS', 'SpecialNotes', 'EyeValidationError.z'], axis=1)
    return df
    #display(df)

In [3]:
def checkIncomplete(path,df):
    des_index = []
    for i in range(0,df['ParticipantUuid'].count()):
        name = df['ParticipantUuid'][i]
        Liste = SelectAllFilesOfSameName(name,path)
        if len(Liste)>4:
            des_index.append(i)
    df = df.iloc[des_index] 
    df = df.reset_index(drop=True)
    return(df)

In [4]:
def selectConditions(df, ConditionName):
    df = df[(df.ExperimentalCondition == ConditionName)]#df[(df.ExperimentalCondition == 'Autonomous')] |]
    df = df.reset_index(drop=True)
    return df

In [5]:
def SelectAllFilesOfSameName(name,path):
    files = glob.glob(f'{path}/*.txt')
    mylist = list()
    for fi in files[:]: # reading all files
        if name in fi:
            mylist.append(fi)                            
    return mylist

In [6]:
def SelectAllCSVOfSameName(path):
    files = glob.glob(f'{path}/*.csv')
    mylist = list()
    for fi in files[:]: # reading all files
        mylist.append(fi)                            
    return mylist

In [7]:
ToDrop = ['FPS','pupilDiameterMillimetersLeft', 'pupilDiameterMillimetersRight',
   'EyeBlinkingWorldLeftTobii', 'EyeBliningWorldRightTobii',
   'EyeBlinkingLocalLeftTobii', 'EyeBlinkingLocalRightTobii','hitObjects','HmdRotation_x', 'HmdRotation_y',
   'HmdRotation_z', 'HmdRotation_w','EyePositionLeftWorld_x','EyePositionLeftWorld_y', 'EyePositionLeftWorld_z',
   'EyeDirectionLeftWorld_x', 'EyeDirectionLeftWorld_y',
   'EyeDirectionLeftWorld_z', 'EyePositionLeftLocal_x',
   'EyePositionLeftLocal_y', 'EyePositionLeftLocal_z',
   'EyeDirectionLeftLocal_x', 'EyeDirectionLeftLocal_y',
   'EyeDirectionLeftLocal_z', 'EyePositionRightWorld_x',
   'EyePositionRightWorld_y', 'EyePositionRightWorld_z',
   'EyePositionRightLocal_x', 'EyePositionRightLocal_y',
   'EyePositionRightLocal_z', 'EyeDirectionRightWorld_x',
   'EyeDirectionRightWorld_y', 'EyeDirectionRightWorld_z',
   'EyeDirectionRightLocal_x', 'EyeDirectionRightLocal_y',
   'EyeDirectionRightLocal_z', 'EyePositionWorldCombinedTobii_x',
   'EyePositionWorldCombinedTobii_y', 'EyePositionWorldCombinedTobii_z',
   'EyePositionLocalCombinedTobii_x', 'EyePositionLocalCombinedTobii_y',
   'EyePositionLocalCombinedTobii_z', 'EyeDirectionWorldCombinedTobii_x',
   'EyeDirectionWorldCombinedTobii_y', 'EyeDirectionWorldCombinedTobii_z',
   'EyeDirectionLocalCombinedTobii_x', 'EyeDirectionLocalCombinedTobii_y',
   'EyeDirectionLocalCombinedTobii_z']
ToDrop2 = [ 'UnixTimeStamp', 'TobiiTimeStamp',
       'EyeOpennessLeftSranipal', 'EyeOpennessRightSranipal', 'HmdPosition_x',
       'HmdPosition_y', 'HmdPosition_z', 'NoseVector_x', 'NoseVector_y',
       'NoseVector_z', 'EyePositionCombinedWorld_x',
       'EyePositionCombinedWorld_y', 'EyePositionCombinedWorld_z', 'EyePositionCombinedLocal_x',
       'EyePositionCombinedLocal_y', 'EyePositionCombinedLocal_z',
       'CarRotation_x', 'CarRotation_y', 'CarRotation_z',
       'CarRotation_w',
       'EyeDirectionCombinedLocal_x_copy', 'EyeDirectionCombinedLocal_y_copy',
       'EyeDirectionCombinedLocal_z_copy', 'is_blink']

In [8]:
def LoadEyetrackingData(mylist):
    start = time.process_time()
    et_df = pd.DataFrame()
    #print(" Before Reading :",time.process_time() - start)
    for fi in mylist:
        with open(fi, 'r') as j:
            tmp = json.loads(j.read())
        for nS, sample in enumerate(tmp):
            tmpdf = pd.json_normalize(data=sample, sep='_')
            
            tmpdf = tmpdf.drop(ToDrop,axis=1)
    
            et_df = pd.concat([et_df, tmpdf], ignore_index=True)

    if not et_df.empty:
        et_df = et_df.drop_duplicates()
        et_df['uid'] = fi.split('\\')[-1].split('_')[0]
        et_df['timestamp'] = pd.to_datetime(et_df.UnixTimeStamp, utc=True, unit='s', origin='unix')
        et_df = et_df.sort_values(by=['timestamp'])
        et_df = et_df.reset_index(drop=True)
        et_df.set_index('timestamp', inplace=True)
        #print(" Before Timestamp :",time.process_time() - start)
        et_df['time_from_start'] = (et_df
    #                                    .reset_index()
    #                                   .groupby(['uid','scene',], as_index=False)
    #                                    .timestamp
                                   .transform(lambda x: 
                                              (x.index - x.index[0])/np.timedelta64(1,'s'))
                                   .UnixTimeStamp
                                   )
        et_df = et_df.reset_index()
        et_df[['EyeDirectionCombinedLocal_x_copy', 'EyeDirectionCombinedLocal_y_copy', 'EyeDirectionCombinedLocal_z_copy']]=et_df[['EyeDirectionCombinedLocal_x', 'EyeDirectionCombinedLocal_y', 'EyeDirectionCombinedLocal_z']]
        et_df['is_blink'] = ((et_df['EyeOpennessLeftSranipal'] < 0.1)&(et_df['EyeOpennessRightSranipal'] < 0.1))
        et_df.loc[(et_df['is_blink'] == True),['EyeDirectionCombinedLocal_x_copy', 'EyeDirectionCombinedLocal_y_copy', 'EyeDirectionCombinedLocal_z_copy']] = np.nan

        #print(" Before Interpolation :",time.process_time() - start)
        tmpdf = (et_df
                 .groupby(['uid'])['EyeDirectionCombinedLocal_x_copy', 'EyeDirectionCombinedLocal_y_copy', 'EyeDirectionCombinedLocal_z_copy']
                 .transform(lambda x: 
                 x.interpolate(method='polynomial', order=3, limit_direction ='both'))
                 .reset_index()
                )
        et_df[['combinedEyeDir_x_intp', 'combinedEyeDir_y_intp', 'combinedEyeDir_z_intp']] = tmpdf[['EyeDirectionCombinedLocal_x_copy', 'EyeDirectionCombinedLocal_y_copy', 'EyeDirectionCombinedLocal_z_copy']]
        #et_df = et_df.drop(ToDrop2,axis=1)  
        #print(" After Interpolation :",time.process_time() - start)
        return et_df
        #return et_df

In [10]:
def ReadSteeringData(mylist):
    start = time.process_time()
    et_df = pd.DataFrame()
    for fi in mylist:
        with open(fi, 'r') as j:
            tmp = json.loads(j.read())
        for nS, sample in enumerate(tmp):
            tmpdf = pd.json_normalize(data=sample, sep='_')
            et_df = pd.concat([et_df, tmpdf], ignore_index=True)
            
    
    if not et_df.empty:
        et_df = et_df.drop_duplicates()
        et_df['uid'] = fi.split('\\')[-1].split('_')[0]
        et_df['timestamp'] = pd.to_datetime(et_df.TimeStamp, utc=True, unit='s', origin='unix')
        et_df = et_df.sort_values(by=['timestamp'])
        et_df = et_df.reset_index(drop=True)
        et_df = et_df.reset_index(drop=True)
        
        #print(" Steering Reading :",time.process_time() - start)
        return et_df



In [11]:
def CombineSteeringAndET(et_df,st_df):
    
    
    st_df.set_index('timestamp', inplace=True)
    
    #Check if DF are equally long
    b = st_df.shape[0]- et_df.shape[0]
    st_df.drop(st_df.tail(b).index,inplace=True)

    st_df = st_df.reset_index()
    
    et_df['Equal'] = np.where(et_df['timestamp'] == st_df['timestamp'],True,False)
    et_df[['SteeringInput', 'AcellerationInput', 'BrakeInput']] = st_df[['SteeringInput', 'AcellerationInput', 'BrakeInput']]
    return et_df


In [13]:
def ReadEventData(mylist):
    interims_df = pd.DataFrame()
    Event_dur_df = pd.DataFrame()
    Event_object_movement_df = pd.DataFrame()
    event_samples_df = pd.DataFrame()
    for fi in mylist:
        with open(fi, 'r') as j:
            tmp = json.loads(j.read())
            x = tmp.get('EventBehavior')
            for p in range(len(x)):
                tmpdf = pd.json_normalize(data=x[p], sep='_')
                interims_df = tmpdf[['EventName', 'StartofEventTimeStamp', 'EndOfEventTimeStamp',
           'EventDuration', 'SuccessfulCompletionState', 'HitObjectName']]
                Event_dur_df = pd.concat([Event_dur_df,interims_df],ignore_index = True)
                tmpdf2 = pd.json_normalize(tmpdf["dynamicObjectData"])
                for nS, sample in enumerate(tmpdf2):
                    temp = list(tmpdf2[nS])
                    tmpdf3 = pd.json_normalize(data=temp, sep='_')
                    tmpdf3['uid'] = fi.split('\\')[-1].split('_')[0]
                    tmpdf3['scene'] = fi.split('_')[-1].split('.')[0]
                    tmpdf3[['EventName', 'StartofEventTimeStamp', 'EndOfEventTimeStamp',
           'EventDuration', 'SuccessfulCompletionState', 'HitObjectName']] = interims_df[['EventName', 'StartofEventTimeStamp', 'EndOfEventTimeStamp',
           'EventDuration', 'SuccessfulCompletionState', 'HitObjectName']]
                    event_samples_df = pd.concat([event_samples_df, tmpdf3], ignore_index=True)
    if not event_samples_df.empty:
        event_samples_df = event_samples_df.drop_duplicates()
        
        event_samples_df['timestamp'] = pd.to_datetime(event_samples_df.UnixTimeStamp, utc=True, unit='s', origin='unix')
        event_samples_df['StartofEventTime'] = pd.to_datetime(event_samples_df.StartofEventTimeStamp, utc=True, unit='s', origin='unix')
        event_samples_df['EndofEventTime'] = pd.to_datetime(event_samples_df.EndOfEventTimeStamp, utc=True, unit='s', origin='unix')
        event_samples_df = event_samples_df.sort_values(by=['timestamp'])
        
        Event_object_movement_df = event_samples_df[['uid','EventName','timestamp','ObjectName0', 'ObjectName1', 'ObjectName2',
       'ObjectPosition0.x', 'ObjectPosition0.y', 'ObjectPosition0.z','ObjectPosition1.x', 'ObjectPosition1.y',
       'ObjectPosition1.z','ObjectPosition2.x', 'ObjectPosition2.y', 'ObjectPosition2.z']]
        
        event_samples_df = event_samples_df.drop(['UnixTimeStamp', 'ObjectName0', 'ObjectName1', 'ObjectName2',
       'ObjectPosition0.x', 'ObjectPosition0.y', 'ObjectPosition0.z','timestamp',
       'ObjectRotation0.x', 'ObjectRotation0.y', 'ObjectRotation0.z',
       'ObjectRotation0.w', 'ObjectPosition1.x', 'ObjectPosition1.y',
       'ObjectPosition1.z', 'ObjectRotation1.x', 'ObjectRotation1.y',
       'ObjectRotation1.z', 'ObjectRotation1.w', 'ObjectPosition2.x',
       'ObjectPosition2.y', 'ObjectPosition2.z', 'ObjectRotation2.x',
       'ObjectRotation2.y', 'ObjectRotation2.z', 'ObjectRotation2.w',  'StartofEventTimeStamp', 'EndOfEventTimeStamp',
        'HitObjectName'
       ],axis=1)
        
        
        #event_samples_df.to_csv(path)
        #event_samples_df = event_samples_df[(event_samples_df.EventName == 'Event_2')|(event_samples_df.EventName == 'Event_6')|(event_samples_df.EventName == 'Event_8')|(event_samples_df.EventName == 'Event_10')]
        Event_object_movement_df = Event_object_movement_df.drop_duplicates().reset_index(drop=True)
        event_samples_df = event_samples_df.drop_duplicates().reset_index(drop=True)
        #Event_object_movement_df.to_csv(path2)
        return(event_samples_df,Event_object_movement_df)

In [14]:
def EyeThetaCalculation(et_samples_df):
    et_samples_df.reset_index(inplace=False)
    et_samples_df['eye_theta_h'] = np.arctan2(et_samples_df['EyeDirectionCombinedLocal_x'], 
                                           et_samples_df['EyeDirectionCombinedLocal_z'])
    et_samples_df['eye_theta_v'] = np.arctan2(et_samples_df['EyeDirectionCombinedLocal_y'], 
                                           et_samples_df['EyeDirectionCombinedLocal_z'])
    et_samples_df['eye_theta_h_int'] = np.arctan2(et_samples_df['combinedEyeDir_x_intp'], 
                                           et_samples_df['combinedEyeDir_z_intp'])
    et_samples_df['eye_theta_v_int'] = np.arctan2(et_samples_df['combinedEyeDir_y_intp'], 
                                           et_samples_df['combinedEyeDir_z_intp'])
    return et_samples_df

In [15]:
def EyeThetaCalculationWorld(et_samples_df):
    et_samples_df.reset_index(inplace=False)
    et_samples_df['eye_theta_h_World'] = np.arctan2(et_samples_df['EyeDirectionCombinedWorld_x'], 
                                           et_samples_df['EyeDirectionCombinedWorld_z'])
   
    return et_samples_df

In [17]:
def CalculateEventTime(et_df,et_dur_df):
    Start = et_df.iloc[0]['timestamp']
    for t in range(0,et_dur_df['uid'].count()):
        et_dur_df['StartofEventTime'][t]=(pd.to_datetime(et_dur_df['StartofEventTime'][t])-pd.to_datetime(Start)).total_seconds()
        et_dur_df['EndofEventTime'][t]=(pd.to_datetime(et_dur_df['EndofEventTime'][t])-pd.to_datetime(Start)).total_seconds()

    #display(et_dur_df)
    return et_dur_df

In [18]:
def CutDatatoEvents(df,evt_df,mylist,EVTpath,mvmt_df):
    for index, row in evt_df.iterrows():
        #print(df)
        #print(evt_df)
        if row['SuccessfulCompletionState']:
            Event_O_df = pd.DataFrame()
            EventStart = row['StartofEventTime'] -1
            EventMvmStart = row['StartofEventTime']
            EventEnd = row['EndofEventTime'] + 1 
            EventMvmEnd = row['EndofEventTime']
            Start_index = df['time_from_start'].sub(EventStart).abs().idxmin()
            End_index = df['time_from_start'].sub(EventEnd).abs().idxmin()
            Start_index_mvm = df['time_from_start'].sub(EventMvmStart).abs().idxmin()
            End_index_mvm = df['time_from_start'].sub(EventMvmEnd).abs().idxmin()
            EventName = row['EventName']

            path = os.path.join(EVTpath,EventName+'.csv')

            if "..\\Only_Events\\"+EventName+".csv" in mylist[:]:
                Event_O_df = pd.read_csv(path)
                print("Found file ", EventName)
            else :
                Event_O_df = pd.DataFrame()
                print("New File")
            new_df1 = df.loc[Start_index:Start_index_mvm]
            
            new_df1.loc[:, "EventTime"] = new_df1["time_from_start"].apply(lambda x: x - (EventStart+1))
            
            new_df2 = df.loc[Start_index_mvm:End_index_mvm]
            
            new_df2.loc[:, "EventTime"] = new_df2["time_from_start"].apply(lambda x: x - (EventStart+1))
            #print("df2" , new_df2.shape)
            evt_df2 = mvmt_df.loc[mvmt_df['EventName'] == EventName]
            new_df2 = pd.merge(new_df2,evt_df2,how = 'left',on = 'timestamp',suffixes = ['','_'])
            
            #print("mvm",evt_df2.shape )
            #print("newNewdf2", new_df2.shape)
            new_df3 = df.loc[End_index_mvm:End_index]
            
            new_df3.loc[:, "EventTime"] = new_df3["time_from_start"].apply(lambda x: x - (EventStart+1))
            
            new_df = pd.concat([new_df1,new_df2,new_df3], ignore_index=True)
            Event_O_df = pd.concat([Event_O_df, new_df], ignore_index=True)
            

            Event_O_df.to_csv(path, index = False)


In [21]:
def GetAverage(df,stringD,binS):
    
    df = df.sort_values(by = stringD)
    
    df['bins'] = pd.cut(
        df[stringD], 
        bins=np.arange(df[stringD].iloc[0],df[stringD].iloc[-1]+binS,binS),
        labels=np.arange(df[stringD].iloc[0]+binS,df[stringD].iloc[-1]+binS,binS)
        )
    df['bins'] = df['bins'].astype(float)
    df4 = df.groupby('bins').agg({
        #'RoadAngle':np.nanmean,
        'SteeringInput':np.nanmean,
        'AcellerationInput':np.nanmean,
        'BrakeInput':np.nanmean,
        'time_from_start':np.nanmean,
        'CarPosition_x':np.nanmean,
        'CarPosition_z':np.nanmean,
        'eye_theta_h':np.nanmean,
        'eye_theta_h_HEAD':np.nanmean,
        'eye_theta_h_World':np.nanmean,
        'AutoRotation_R_Y':np.nanmean,
        'Head_theta_h':np.nanmean,
        'nose_theta_h':np.nanmean
        }).reset_index()
    return df4

In [22]:
def CheckForSteeringNulls(df):
    uids = list(df.uid.unique())
    for u in uids:
        df2 = df.loc[df['uid'] == u]
        df.loc[df['uid'] == u, ['SteeringInput_']] = ((df2['SteeringInput']-df2['SteeringInput'].mean())/df2['SteeringInput'].std())
        unique = list(df2.SteeringInput.unique())
        if(len(unique)<2):
            df.loc[df['uid'] == u, ['SteeringInput']] = np.nan
        
    return df

In [23]:
def Distance(df):
    #classes = list(df.uid.unique())
    #for c in classes[:]:
    #print(df.shape)
    point2 = np.array((df['CarPosition_x'].values[0], df['CarPosition_y'].values[0], df['CarPosition_z'].values[0]))

    distance = list()


    summi = 0
    #print("df.shape")
    for index, row in df.iterrows():
        point1 = np.array((row['CarPosition_x'], row['CarPosition_y'], row['CarPosition_z']))
        temp = point1 - point2
        sum_sq = np.dot(temp.T, temp)
        distance.append( np.sqrt(sum_sq)+summi)
        summi =  np.sqrt(sum_sq)+summi
        point2 = point1
    #print(sum(distance))
    #print(distance)
    df['Distance'] = np.array(distance)
    #print(len(distance))

    #df2['Distance']= np.array(distance)
    return df

In [26]:
def RotationStuff(df_A):   

    df_B = pd.DataFrame()
    # We want the head rotation horizontal
    # we have the eyedirection_world, car rotation and eye direction local
    # headrotation should be the eyedirection world minus car rotation and minus eye local
    # car rotation is in quaternion
    # eyedirection world is euler so is eyelocal

    #Car rotation to Rotation Matrix
    df_B['CarRotation_all'] = df_A[[
    'CarRotation_x', 'CarRotation_y', 'CarRotation_z', 'CarRotation_w'
    ]].apply(lambda r: tuple(r), axis=1).apply(np.array)
    df_B['AutoRotation_R'] = df_B['CarRotation_all'].apply(R)
    df_B['AutoRotation_R_euler'] = df_B['AutoRotation_R'].apply(Euler)
    #df_B['AutoRotation_R_euler_Z'] = df_B['AutoRotation_R_euler'].apply(lambda x: x[0])
    #df_B['AutoRotation_R_euler_Y'] = df_B['AutoRotation_R_euler'].apply(lambda x: x[1])
    #df_B['AutoRotation_R_euler_X'] = df_B['AutoRotation_R_euler'].apply(lambda x: x[2])

    df_B['EyeDirectionCombinedWorld'] = df_A[['EyeDirectionCombinedWorld_z', 'EyeDirectionCombinedWorld_y',
       'EyeDirectionCombinedWorld_x'
    ]].apply(lambda r: tuple(r), axis=1).apply(np.array)
    df_B['Combined_Nose'] = df_A[['NoseVector_z', 'NoseVector_y',
       'NoseVector_x'
    ]].apply(lambda r: tuple(r), axis=1).apply(np.array)
    df_B['AutoRotation_R_inv'] = (df_B['AutoRotation_R'].apply(lambda x: x.inv()))
    df_B['New_Nose'] = df_B['AutoRotation_R_inv']
    df_B['New_Direction_2'] = df_B['AutoRotation_R']
    for index, row in df_B.iterrows():
        #row['New_Direction']= (row['AutoRotation_R_inv'].apply(row['EyeDirectionCombinedWorld']))
        row['New_Direction_2']= (row['AutoRotation_R'].apply(row['EyeDirectionCombinedWorld']))
        row['New_Nose']=(row['AutoRotation_R'].apply(row['Combined_Nose']))

    df_B['HeadRotation_R_euler_Z'] = df_B['New_Direction_2'].apply(lambda x: x[0])
    df_B['HeadRotation_R_euler_Y'] = df_B['New_Direction_2'].apply(lambda x: x[1])
    df_B['HeadRotation_R_euler_X'] = df_B['New_Direction_2'].apply(lambda x: x[2])
    df_B['Nose_euler_Z'] = df_B['New_Nose'].apply(lambda x: x[0])
    df_B['Nose_euler_Y'] = df_B['New_Nose'].apply(lambda x: x[1])
    df_B['Nose_euler_X'] = df_B['New_Nose'].apply(lambda x: x[2])

    df_B.reset_index(inplace=False)
    df_A['eye_theta_h_HEAD'] = np.arctan2(df_B['HeadRotation_R_euler_X'], 
                                           df_B['HeadRotation_R_euler_Z'])
    df_A['eye_theta_h_World'] = np.arctan2(df_A['EyeDirectionCombinedWorld_x'], 
                                           df_A['EyeDirectionCombinedWorld_z'])
    
    df_A['nose_theta_h'] = np.arctan2(df_B['Nose_euler_X'], 
                                           df_B['Nose_euler_Z'])
    df_A['AutoRotation_R_Y'] = df_B['AutoRotation_R_euler'].apply(lambda x: x[1])
    df_A['Head_theta_h'] = df_A['eye_theta_h_HEAD'] - df_A['eye_theta_h'] 
    df_A = ThetaWorldToNan(df_A,'Head_theta_h') 
    df_A = ThetaWorldToNan(df_A,'eye_theta_h_HEAD') 
    df_A = ThetaWorldToNan(df_A,'eye_theta_h') 
    df_A = ThetaWorldToNan(df_A,'eye_theta_h_World')
    
    """
    #Eyedirection_world to TM

    df_B['EyeDirectionCombinedWorld_R'] = df_B['EyeDirectionCombinedWorld'].apply(From_Euler)

    #Eyedirection_Local to TM

    df_B['EyeDirectionCombinedLocal'] = df_A[['EyeDirectionCombinedLocal_z', 'EyeDirectionCombinedLocal_y',
       'EyeDirectionCombinedLocal_x'
    ]].apply(lambda r: tuple(r), axis=1).apply(np.array)
    df_B['EyeDirectionCombinedLocal_R'] = df_B['EyeDirectionCombinedLocal'].apply(From_Euler)

    df_B['PureHead'] = df_B['EyeDirectionCombinedWorld_R']*(df_B['AutoRotation_R'].apply(lambda x: x.inv()))*(df_B['EyeDirectionCombinedLocal_R'].apply(lambda x: x.inv()))


    #Car rotation to Euler
    df_B['HeadRotation_R_euler'] = df_B['PureHead'].apply(Euler)

    ### spliting zyx values
    df_B['HeadRotation_R_euler_Z'] = df_B['HeadRotation_R_euler'].apply(lambda x: x[0])
    df_B['HeadRotation_R_euler_Y'] = df_B['HeadRotation_R_euler'].apply(lambda x: x[1])
    df_B['HeadRotation_R_euler_X'] = df_B['HeadRotation_R_euler'].apply(lambda x: x[2])

    df_B.reset_index(inplace=False)
    df_A['eye_theta_h_HEAD'] = np.arctan2(df_B['HeadRotation_R_euler_X'], 
                                           df_B['HeadRotation_R_euler_Z'])

    df_A['eye_theta_h_Car'] = np.arctan2(df_B['AutoRotation_R_euler_X'], 
                                           df_B['AutoRotation_R_euler_Z'])
    df_B['time_from_start'] = df_A['time_from_start']"""
    #print(df_A.columns)
    #display(df_A)
    return df_A

In [27]:
#Function for converting t o Euler angles

def Euler(x):
    from scipy.spatial.transform import Rotation as R
    Euler_angles = x.as_euler('zyx', degrees=True)
    return Euler_angles


def From_Euler(x):
    from scipy.spatial.transform import Rotation as R
    R = R.from_euler('zyx', x, degrees=True)
    return R


def R(x):
    from scipy.spatial.transform import Rotation as R
    R = R.from_quat(x)
    return R
